# AMPL Capacitated p-Median Problem with GCG
[![cpmp.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/authors/lentz/gcg/cpmp.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/lentz/gcg/cpmp.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/authors/lentz/gcg/cpmp.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/authors/lentz/gcg/cpmp.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/authors/lentz/gcg/cpmp.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/authors/lentz/gcg/cpmp.ipynb)](https://colab.ampl.com)

Description: Dantzig-Wolfe decomposition for Capacitated p-Median Problem with GCG

Tags: GCG, cpmp, amplpy, dantzig-wolfe decomposition, branch-price-and-cut, highlights

Notebook author: Jurgen Lentz <<jurgenlentz26@gmail.com>>

In [1]:
# Install dependencies
%pip install -q amplpy

In [2]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["gcg"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

## Capacitated $p$-median problem

The goal of the capacitated $p$-median problem (CPMP) is to fix $p \in \mathbb{N}$ locations as medians from a given number of $n \in \mathbb{N}$ locations. Furthermore, we are given demands $q_{i} \geq 0$ and capacities $Q_{i} \geq 0$ for every location $i \in \{1,\ldots,n\}$ as well as distances $d_{ij} \geq 0$ for each pair of locations $i,j \in \{1,\ldots,n\}$. While fixing the medians, we also assign every location $i \in \{1,\ldots,n\}$ to exactly one median such that the demands of all locations assigned to the median can be satisfied by the capacity of the median. An assignment of locations to a median is called a cluster. The overall objective is to minimize the total distance between the medians and their assigned locations.

CPMP can be modeled as follows (see [Ceselli et al.](http://yalma.fime.uanl.mx/~roger/work/teaching/class_tso/docs_project/problems/CPMP/2005-nets-A%20Branch-and-Price%20Algorithm%20for%20the%20Capacitated%20p-Median%20Problem.pdf)):

$$
\begin{aligned}
\text{minimize} \quad &\sum_{i = 1}^{n} \sum_{j = 1}^{n} d_{i j}  x_{i j} \\
\text{subject to} \quad &\sum_{j = 1}^{n} x_{i j} = 1 \quad \forall i \in \{1,...,n\} \\
&\sum_{i = 1}^{n} q_{i} x_{i j} \leq Q_{j} y_{j} \quad \forall j \in \{1,...,n\} \\
&\sum_{j = 1}^{n} y_{j} = p \\
&x_{i j} \in \{0,1\} \quad \forall i,j \in \{1,...,n\} \\
&y_{j} \in \{0,1\} \quad \forall j \in \{1,...,n\}
\end{aligned}
$$

We will solve an CPMP instance using Dantzig-Wolfe decomposition with the GCG solver in AMPL. Therefore, we first model CPMP in AMPL (shown below).

In [3]:
%%ampl_eval
param n;
param p;

set I = 1..n ordered;
set J = 1..n ordered;

param d {I,J};
param w {I};
param C {J};

var x {I,J} binary;
var y {J} binary;

minimize Cost:  sum {i in I} sum {j in J} d[i,j] * x[i,j];

subject to Allocate {i in I}:
   sum {j in J} x[i,j] = 1;

subject to Capacity {j in J}:
   sum {i in I} w[i] * x[i,j] <= C[j] * y[j];

subject to NFacilities:
  sum{j in J} y[j] <= p;

We generate a small instance with 5 locations and 2 locations as medians.

In [4]:
ampl.param["n"] = 5
ampl.param["p"] = 2

d = [
    [0, 6, 54, 52, 19],
    [6, 0, 28, 75, 61],
    [54, 28, 0, 91, 40],
    [52, 75, 91, 0, 28],
    [19, 61, 40, 28, 0],
]
ampl.param["d"] = {(i, j): d[i - 1][j - 1] for i in range(1, 6) for j in range(1, 6)}

ampl.param["w"] = [14, 13, 9, 15, 6]
ampl.param["C"] = [39, 39, 39, 39, 39]

## Automatic Mode in GCG with AMPL

We use AMPL to call the solver GCG to solve our CPMP instance automatically without providing any information about the Dantzig-Wolfe decomposition. Here, GCG detects different decompositions and chooses heuristically the best decomposition. Afterwards, the solver uses a branch-price-and-cut algorithm to solve it to optimality.

In [5]:
ampl.solve(solver="gcg", gcg_options="outlev=1")
assert ampl.solve_result == "solved"

GCG 4.0.0:   tech:outlev = 1
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 10 upgd conss, 0 impls, 5 clqs
(round 2, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 11 upgd conss, 0 impls, 30 clqs
   (0.0s) probing cycle finished: starting next cycle
presolving (3 rounds: 3 fast, 3 medium, 3 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 30 cliques
presolved problem has 30 variables (30 bin, 0 int, 0 impl, 0 cont) and 11 constraints
      6 constraints of type <knapsack>
      5 constraints of type <setppc>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00
 Consclassifier "nonzeros" yields a classification with 2  different constraint classes 
 Consclassifier "constypes" yields a classification with 2 different constraint classes 
 

## Using a custom decomposition in GCG with AMPL

AMPL allows the users to create their own decomposition and forwards it to GCG using suffixes. Here, we assign the allocation and pmedian constraints to the master/linking constraints and each capacity constraint to a different pricing problem.

In [6]:
%%ampl_eval

suffix master IN, binary;
suffix block IN, integer;

let {i in I}
   Allocate[i].master := 1;

let NFacilities.master := 1;

let {j in J}
   Capacity[j].block := j;

In [7]:
ampl.solve()
assert ampl.solve_result == "solved"

GCG 4.0.0:   tech:outlev = 1
 added complete decomp for original problem with 5 blocks and 6 masterconss, 0 linkingvars, 0 mastervars, and max white score of   0.363636 
presolving:
(round 1, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 10 upgd conss, 0 impls, 5 clqs
(round 2, exhaustive) 0 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 11 upgd conss, 0 impls, 30 clqs
   (0.0s) probing cycle finished: starting next cycle
presolving (3 rounds: 3 fast, 3 medium, 3 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 30 cliques
presolved problem has 30 variables (30 bin, 0 int, 0 impl, 0 cont) and 11 constraints
      6 constraints of type <knapsack>
      5 constraints of type <setppc>
transformed objective value is always integral (scale: 1)
Presolving Time: 0.00
 calculated translation; number of mis

Analogously, the user can fix constraints as pricing block or master constraint and variables as pricing block, master or linking variables. It is not needed to provide a complete decomposition. If the user provides a partial decomposition, GCG completes the decomposition by fixing only the left constraints and variables using its detection loop.

**_NOTE:_**  The index of pricing block constraints/variables starts at 1.